#### ! 선행 설치 모듈 및 라이브러리

In [1]:
%pip install seaborn
%pip install nltk
%pip install googletrans==4.0.0-rc1

Note: you may need to restart the kernel to use updated packages.
Note: you may need to restart the kernel to use updated packages.
Note: you may need to restart the kernel to use updated packages.


In [2]:
import os
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
import keras
import sklearn
import csv
import re
import nltk
import hashlib
import pickle
from googletrans import Translator
from nltk.sentiment.vader import SentimentIntensityAnalyzer
from nltk.corpus import stopwords
from nltk.tokenize import word_tokenize
from tqdm import tqdm
import time
nltk.download('vader_lexicon')
nltk.download('punkt')
nltk.download('stopwords')

[nltk_data] Downloading package vader_lexicon to
[nltk_data]     C:\Users\정재현\AppData\Roaming\nltk_data...
[nltk_data]   Package vader_lexicon is already up-to-date!
[nltk_data] Downloading package punkt to
[nltk_data]     C:\Users\정재현\AppData\Roaming\nltk_data...
[nltk_data]   Package punkt is already up-to-date!
[nltk_data] Downloading package stopwords to
[nltk_data]     C:\Users\정재현\AppData\Roaming\nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


True

# 1. 데이터 확인 및 전처리

먼저 컬럼별 어떤 데이터를 넣어놨고 데이터에 오류나 결측치가 있는지 확인

이후 데이터의 type이나 기입되어 있는 데이터들이 어떤 형식으로 되어있는지 확인한 다음 데이터 전처리를 실시

In [3]:
df = pd.read_csv("./amazon_uk_shoes_products_dataset_2021_12.csv", encoding='utf-8') 
df.head()

,url,product_name,reviewer_name,review_title,review_text,review_rating,verified_purchase,review_date,helpful_count,uniq_id,scraped_at
0,https://www.amazon.co.uk/dp/B07SBX32T5,Klasified Women's Transparent Clear Sneaker Sh...,Jocelyn McSayles,Love em,Love these. Was looking for converses and thes...,5.0,True,Reviewed in the United States on 2 June 2020,2 people found this helpful,36eae4e5-2894-5279-a0b7-d2b330e2b814,24/12/2021 02:26:25
1,https://www.amazon.co.uk/dp/B07SBX32T5,Klasified Women's Transparent Clear Sneaker Sh...,Kenia Rivera,The plastic ripped,"The shoes are very cute, but after the 2nd day...",2.0,True,Reviewed in the United States on 28 October 2021,NaN,f4778bb8-3070-5cb1-b5aa-ffce41a97b57,24/12/2021 02:26:25
2,https://www.amazon.co.uk/dp/B07SBX32T5,Klasified Women's Transparent Clear Sneaker Sh...,Chris Souza,Good quality,Good quality,5.0,True,Reviewed in the United States on 20 January 2021,NaN,db5a7525-d40b-5265-84d8-df4f29837a3b,24/12/2021 02:26:25
3,https://www.amazon.co.uk/dp/B07SBX32T5,Klasified Women's Transparent Clear Sneaker Sh...,Amazon Customer,Good,Great,5.0,True,Reviewed in the United States on 22 April 2021,NaN,75a42851-6462-54b5-988a-27d336221943,24/12/2021 02:26:25
4,https://www.amazon.co.uk/dp/B08SW434MG,"GUESS Women's Bradly Gymnastics Shoe, White, 7 UK",Graziella,PERFETTE!!,Ho scelto il modello bianco con rifinitura die...,5.0,True,Reviewed in Italy on 2 April 2021,2 people found this helpful,232dee43-849e-5d06-ba05-efb3f4814714,24/12/2021 02:26:25


In [4]:
# 로드된 데이터 프레임의 형태를 출력합니다 (형태는 행과 열의 수입니다)
print("데이터의 형태 (행, 열)는 " + str(df.shape) + " 입니다.")

데이터의 형태 (행, 열)는 (6823, 11) 입니다.


In [5]:
# 파일 경로
file_path = './amazon_uk_shoes_products_dataset_2021_12.csv'

# 첫 줄을 읽어서 컬럼 레이블을 확인합니다
with open(file_path, mode='r', encoding='utf-8') as file:
    reader = csv.reader(file)
    column_labels = next(reader)

# 컬럼 레이블을 한 줄씩 출력합니다
formatted_labels = "\n\n".join(column_labels)
print(formatted_labels)

url

product_name

reviewer_name

review_title

review_text

review_rating

verified_purchase

review_date

helpful_count

uniq_id

scraped_at


컬럼은 위와 같은 형태로 존재하는데 저 중에서 필요한 데이터는 사실상 review_title, review_text, review_rating라고 판단

review_rating은 맥락이 필요한 텍스트(비꼬는 리뷰)에 필요할 수 있어서 남겨두었음

helpful_count가 물건에 대한 설명과 감정이 잘 드러난 데이터에 주로 많은 경향이 있어서 가중치로 사용할까 했지만 복잡해지고 정확도에 부정적인 영향을 끼칠 수 있기에 우선 제외

### (1) 특수문자 제거, 영어 번역, 불용어 제거, 제목과 본문 통합

원래 계획은 각각의 과정을 따로 시행하려 했지만 번역 과정에서 너무 많은 시간이 소요되는 문제가 발생

또한 6천개가 넘는 데이터를 매번 확인하며 전처리를 하는 것은 비효율적이라고 판단

따라서 모든 데이터 처리를 한번에 시행하며 영어 번역은 시행속도를 올리기 위해 병렬처리

In [7]:
# 불용어 설정
stop_words = set(stopwords.words('english'))

# Google Translator 설정
translator = Translator()

# 번역 캐시 파일 경로
cache_file = './translation_cache.pkl'

# 번역 캐시 로드 또는 초기화
if os.path.exists(cache_file):
    with open(cache_file, 'rb') as f:
        translation_cache = pickle.load(f)
else:
    translation_cache = {}

# 텍스트 해시 함수
def text_hash(text):
    return hashlib.md5(text.encode()).hexdigest()

# 통합된 전처리 함수
def preprocess_and_translate(row):
    # 번역 함수 (캐시 사용 및 딜레이 추가)
    def translate_to_english(text):
        if not isinstance(text, str):
            text = ''
        text_key = text_hash(text)
        if text_key in translation_cache:
            return translation_cache[text_key]
        else:
            try:
                detected_lang = translator.detect(text).lang
                print(f"Detected language: {detected_lang}")
                translated = translator.translate(text, src=detected_lang, dest='en').text
                translation_cache[text_key] = translated
                # 딜레이 추가
                time.sleep(2)  # 2초 딜레이 추가
                return translated
            except Exception as e:
                print(f"Translation error: {e}")
                return text  # 번역 실패 시 원본 텍스트 반환

    # 특수 문자 제거 함수
    def remove_special_characters(text):
        return re.sub(r'[^a-zA-Z0-9\s]', '', text)

    # 텍스트 전처리 함수 (불용어 제거 포함)
    def preprocess_text(text):
        text = text.lower()  # 소문자 변환
        text = re.sub(r'\s+', ' ', text).strip()  # 여러 공백을 하나의 공백으로 변환하고 양쪽 공백 제거
        tokens = word_tokenize(text)
        filtered_tokens = [word for word in tokens if word not in stop_words]
        return ' '.join(filtered_tokens)

    # 제목과 본문 통합
    title = row['review_title'] if pd.notna(row['review_title']) else ''
    review = row['review_text'] if pd.notna(row['review_text']) else ''
    combined = title + ' ' + review if title and review else title or review

    # 번역
    translated_text = translate_to_english(combined)
    
    # 특수 문자 제거 및 전처리
    translated_text = remove_special_characters(translated_text)
    cleaned_text = preprocess_text(translated_text)
    
    return cleaned_text

# 모든 리뷰 텍스트와 제목을 한 번에 전처리하고 번역 (진행 상황 표시)
tqdm.pandas(desc="Processing reviews")
df['processed_review'] = df.progress_apply(preprocess_and_translate, axis=1)

# 번역 캐시 저장
with open(cache_file, 'wb') as f:
    pickle.dump(translation_cache, f)

# 전처리된 리뷰 텍스트 확인
print(df['processed_review'].head())

# 필요한 컬럼만 선택하여 저장
output_file_path = './processed_reviews.csv'
df[['processed_review', 'review_rating']].to_csv(output_file_path, index=False)

Processing reviews:  14%|█▍        | 984/6823 [00:00<00:03, 1525.05it/s]

Translation error: sequence item 1: expected str instance, NoneType found


Processing reviews:  18%|█▊        | 1220/6823 [00:01<00:04, 1149.59it/s]

Translation error: sequence item 1: expected str instance, NoneType found


Processing reviews:  22%|██▏       | 1535/6823 [00:01<00:04, 1150.28it/s]

...

Translation error: sequence item 1: expected str instance, NoneType found


Processing reviews:  94%|█████████▍| 6430/6823 [00:04<00:00, 2609.93it/s]

Translation error: sequence item 1: expected str instance, NoneType found


Processing reviews: 100%|██████████| 6823/6823 [00:04<00:00, 1465.10it/s]

Translation error: sequence item 2: expected str instance, NoneType found
0    love em love looking converses half price uniq...
1    plastic ripped shoes cute 2nd day wearing tong...
2                            good quality good quality
3                                           good great
4    perfect chose white model finish behind black ...
Name: processed_review, dtype: object
